<center><h1>Exploring the Competitors</h1></center>

### Part 1. Get location data using Foursquare

[Foursquare Places API](https://location.foursquare.com/products/places-api/) is very usefule online application used by many developers & other application like Uber etc. In this project you can used it to retrieve informtion about the places present in the neighborhoods of Toronto. The API returns a JSON file and we need to turn that into a data-frame. Here I’ve chosen similar businesses(pet grooming) for each neighborhood within a radius of 2.5km.

You will need to create an account with Foursquare to access the API.  It is free to sign up, and you get $200 free credit.



In [1]:
import requests
import pandas as pd

In [2]:
toronto_DF = pd.read_csv('toronto_postcode.csv')

In [3]:
display(toronto_DF)

,Postalcode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
...,...,...,...,...,...
98,M9N,43.706876,-79.518188,York,Weston
99,M9P,43.696319,-79.532242,Etobicoke,Westmount
100,M9R,43.688905,-79.554724,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,43.739416,-79.588437,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [4]:
# Set up your API key and other constants for Foursquare API
API_KEY = 'fsq3BRhjBeCdDIuZVVMPdae0pG2+R6EHFyq3p1WiHgnTEjk='  # Replace with your actual Foursquare API key
LIMIT = 20  # Maximum number of venues to return
radius = 2500  # Search radius in meters

# a function to loop through each neighbourhood in the postalcode.csv file, and search for places
def getNearbyVenues(names, postalcode, latitudes, longitudes, radius):
    venues_list = []

    for name, postalcode, lat, lng in zip(names, postalcode, latitudes, longitudes):
        print(f"Processing neighborhood: {name}")

        # Skip if latitude or longitude is missing
        if pd.isnull(lat) or pd.isnull(lng):
            print(f"Skipping {name} due to missing latitude or longitude.")
            continue

        # Create the API request URL and parameters
        # Change the params here to customize your search, https://docs.foursquare.com/developer/reference/place-search
        # The following example looks for pet groomers in the neighbourhood
        # I am searching with categories ID 11134 - pet grooming services
        url = 'https://api.foursquare.com/v3/places/search'
        params = {
            'll': f'{lat},{lng}',
            'radius': radius,
            'limit': LIMIT,
            'categories': 11071
        }

        # Set up headers with the API key
        headers = {
            "Accept": "application/json",
            "Authorization": API_KEY  
        }

        # Make the GET request
        response = requests.get(url, headers=headers, params=params)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to get data for {name}. Status code: {response.status_code}")
            print(f"Error message: {response.text}")
            continue

        results = response.json()

        # Parse the JSON response
        for venue in results.get('results', []):
            # Extract venue information
            venue_name = venue.get('name')
            venue_id = venue.get('fsq_id')
            venue_location = venue.get('geocodes', {}).get('main', {})
            venue_lat = venue_location.get('latitude')
            venue_lng = venue_location.get('longitude')
            venue_categories = venue.get('categories', [])
            venue_category = venue_categories[0]['name'] if venue_categories else 'Unknown'

            # Append to the list
            venues_list.append([
                name,
                postalcode,
                lat,
                lng,
                venue_name,
                venue_id,
                venue_lat,
                venue_lng,
                venue_category
            ])

    # Create a DataFrame from the list
    nearby_venues = pd.DataFrame(venues_list, columns=[
        'Neighborhood',
        'Postalcode',
        'Neighborhood Latitude',
        'Neighborhood Longitude',
        'Venue',
        'fsq_id',
        'Venue Latitude',
        'Venue Longitude',
        'Venue Category'
    ])

    return nearby_venues

# Example usage with your DataFrame 'toronto_DF'
# Ensure 'toronto_DF' has the columns 'Neighborhood', 'Latitude', 'Longitude'
toronto_venues = getNearbyVenues(
    names=toronto_DF['Neighbourhood'],
    postalcode=toronto_DF['Postalcode'],
    latitudes=toronto_DF['Latitude'],
    longitudes=toronto_DF['Longitude'],
    radius=radius
)

# Display the first few rows of the resulting DataFrame
toronto_venues.head()


Processing neighborhood: Malvern, Rouge
Processing neighborhood: Rouge Hill, Port Union, Highland Creek
Processing neighborhood: Guildwood, Morningside, West Hill
Processing neighborhood: Woburn
Processing neighborhood: Cedarbrae
Processing neighborhood: Scarborough Village
Processing neighborhood: Kennedy Park, Ionview, East Birchmount Park
Processing neighborhood: Golden Mile, Clairlea, Oakridge
Processing neighborhood: Cliffside, Cliffcrest, Scarborough Village West
Processing neighborhood: Birch Cliff, Cliffside West
Processing neighborhood: Dorset Park, Wexford Heights, Scarborough Town Centre
Processing neighborhood: Wexford, Maryvale
Processing neighborhood: Agincourt
Processing neighborhood: Clarks Corners, Tam O'Shanter, Sullivan
Processing neighborhood: Milliken, Agincourt North, Steeles East, L'Amoreaux East
Processing neighborhood: Steeles West, L'Amoreaux West
Processing neighborhood: Upper Rouge
Processing neighborhood: Hillcrest Village
Processing neighborhood: Fairview,

,Neighborhood,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",M1B,43.806686,-79.194353,Blue Ocean Nails & Spa,4bb7ab84314e95214481479d,43.801081,-79.199017,Nail Salon
1,"Malvern, Rouge",M1B,43.806686,-79.194353,Nails for you,4c7bfb4993ef236a8bf6b60f,43.806984,-79.222775,Nail Salon
2,"Malvern, Rouge",M1B,43.806686,-79.194353,Nails For You,e72e5651d5c643192ecc9296,43.806573,-79.222242,Nail Salon
3,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497,Jenny's Nail Studio,4c54567da724e21e73793bf6,43.784042,-79.158588,Nail Salon
4,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497,KN'S Nails & Spa,5d80e0dab6277c000866ba5e,43.783375,-79.169641,Nail Salon


In [5]:
# The total number of pet grooming services in Toronto
toronto_venues['Venue'].nunique()

413

In [6]:
# Oberserving the number of competitors in each neighbourhood
toronto_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
Agincourt                                           6
Alderwood, Long Branch                              9
Bathurst Manor, Wilson Heights, Downsview North     9
Bayview Village                                    10
Bedford Park, Lawrence Manor East                  20
                                                   ..
Willowdale, Newtonbrook                            20
Woburn                                              8
Woodbine Heights                                   13
York Mills West                                    16
York Mills, Silver Hills                            6
Name: Venue, Length: 102, dtype: int64

In [7]:
toronto_venues['Venue'].count()

1435

### Part 2. Interactive leaflet map using coordinate data.


In [8]:
import folium # map rendering library

In [9]:
# My example below shows pet groomers in Lawrence Park
target = 'Lawrence Park'

search_area = toronto_venues[toronto_venues['Neighborhood'] == target]
latitude = toronto_DF[toronto_DF['Neighbourhood'] == target]['Latitude']
longitude = toronto_DF[toronto_DF['Neighbourhood'] == target]['Longitude']
display(search_area)

,Neighborhood,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category
475,Lawrence Park,M4N,43.72802,-79.38879,Nailsense Spa,4bdd97185b31c9b6f00d9d16,43.717545,-79.400883,Nail Salon
476,Lawrence Park,M4N,43.72802,-79.38879,Pedimani Nail Spa,4c0d8c0a336220a13aefca77,43.726711,-79.402300,Nail Salon
477,Lawrence Park,M4N,43.72802,-79.38879,Nice Nails and Spa,5040fa9fe4b0133020549852,43.733211,-79.404087,Nail Salon
478,Lawrence Park,M4N,43.72802,-79.38879,Nail Teco,4d44781f7e2e5481eec2608f,43.733458,-79.404548,Nail Salon
479,Lawrence Park,M4N,43.72802,-79.38879,Good Nails,4bcc73cb511f9521bff7b2c7,43.716879,-79.400776,Nail Salon
480,Lawrence Park,M4N,43.72802,-79.38879,Favorite Nail Spa,51bf656e498ecc922cd641eb,43.715587,-79.400008,Nail Salon
481,Lawrence Park,M4N,43.72802,-79.38879,Monluxe Nails and Beauty Bar,5be9e10bdd70c5002ca4cca0,43.712491,-79.399326,Nail Salon
482,Lawrence Park,M4N,43.72802,-79.38879,Nice One Madipedi Spa,4f1f12f2e4b07f86b5b0ab50,43.712072,-79.399681,Nail Salon
483,Lawrence Park,M4N,43.72802,-79.38879,Ananeke Beauty Salon and Spa,53d53d37498e84ef9193df5b,43.709055,-79.398684,Nail Salon
484,Lawrence Park,M4N,43.72802,-79.38879,Classic Nails Salon,519be7af498e0ac086c816b4,43.723447,-79.416294,Nail Salon


In [10]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, venue, neighborhood in zip(search_area['Venue Latitude'], search_area['Venue Longitude'], search_area['Venue'], search_area['Neighborhood']):
    label = '{},{}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

/opt/conda/lib/python3.11/site-packages/folium/utilities.py:94: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
/opt/conda/lib/python3.11/site-packages/folium/utilities.py:100: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
/opt/conda/lib/python3.11/site-packages/folium/utilities.py:102: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(x) for x in coords]


### Part 3. Customer Reviews
It seems like there is one one pet groomer withing 2km of Scarborough Village, I am interested in seeing customer's comments on Funny Bunny.  I can use [Foursquare's Place Tips API.](https://docs.foursquare.com/developer/reference/place-tips)


In [11]:
# a function to loop through the list of pet groomers in the neighborhood and compile all the comments related to them
def getTips(venues, fsq_ids):
    venues_tips = []

    for venue, fsq_id in zip(venues, fsq_ids):
        print(f"Processing venue: {venue}")


        # Create the API request URL and parameters
        url = f"https://api.foursquare.com/v3/places/{fsq_id}/tips"


        # Set up headers with the API key
        headers = {
            "Accept": "application/json",
            "Authorization": API_KEY  
        }
        
        # Make the GET request
        response = requests.get(url, headers=headers)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to get data for {venue}. Status code: {response.status_code}")
            print(f"Error message: {response.text}")
            continue

        results = response.json()
        
        # Parse the JSON response
        for tip in results:
            # Extract venue information
            tip_id = tip.get('id')
            tip_created = tip.get('created_at')
            tip_text = tip.get('text')
   

            # Append to the list
            venues_tips.append([
                venues,
                tip_id,
                tip_created,
                tip_text
            ])
            
    
    # Create a DataFrame from the list
    nearby_tips = pd.DataFrame(venues_tips, columns=[
        'venue_name',
        'id',
        'created_at',
        'text'
    ])

    return nearby_tips


nearby_tips = getTips(
    venues=search_area['Venue'],
    fsq_ids=search_area['fsq_id']
)

# Display the first few rows of the resulting DataFrame
nearby_tips

Processing venue: Nailsense Spa
Processing venue: Pedimani Nail Spa
Processing venue: Nice Nails and Spa
Processing venue: Nail Teco
Processing venue: Good Nails
Processing venue: Favorite Nail Spa
Processing venue: Monluxe Nails and Beauty Bar
Processing venue: Nice One Madipedi Spa
Processing venue: Ananeke Beauty Salon and Spa
Processing venue: Classic Nails Salon
Processing venue: Toronto Nail Studio and Spa
Processing venue: Carnation Nails & Spa
Processing venue: Vida Nails & Spa
Processing venue: Planet Nails
Processing venue: Hoang Thy Nails Inc
Processing venue: Nails For You
Processing venue: Angel Nail & Spa
Processing venue: Aritaum Spa & Nails
Processing venue: Unique Nails N Spa
Processing venue: The Nail Studio


,venue_name,id,created_at,text
0,475 Nailsense Spa 476 ...,5218ea8011d23418834dcdd5,2013-08-24T17:16:48.000Z,Kim is amazing! My best friend introduced me t...
1,475 Nailsense Spa 476 ...,4f1efcd4e4b03f6d8f0a2112,2012-01-24T18:47:48.000Z,This place is amazing. Everytime I go in they ...
2,475 Nailsense Spa 476 ...,4fabfe0be4b09eda9f48c6b9,2012-05-10T17:42:35.000Z,I like Cozy Nails too but I come here now bc t...
3,475 Nailsense Spa 476 ...,4cd08c12adf29c7457e57b95,2010-11-02T22:09:22.000Z,very clean and good nail polish selection but ...
4,475 Nailsense Spa 476 ...,4fcf7b21e4b08ffd9bb87751,2012-06-06T15:45:37.000Z,Thuy does the best Brazilian wax in the city. ...
5,475 Nailsense Spa 476 ...,4c320f566f1fef3b272bec3d,2010-07-05T16:59:02.000Z,Thuy was extremely friendly and professional -...
6,475 Nailsense Spa 476 ...,4ec1e48f0aaf6450bba0b208,2011-11-15T04:03:27.000Z,cheap mani pedis. clean
7,475 Nailsense Spa 476 ...,4bf5536770c603bb77419cb4,2010-05-20T15:21:11.000Z,Thuy is fantastic
8,475 Nailsense Spa 476 ...,5181b06b498efc5a5f060c38,2013-05-02T00:16:43.000Z,These guys know how to treat customers. Total...
9,475 Nailsense Spa 476 ...,51213e66e4b02c9c6811d0a2,2013-02-17T20:32:38.000Z,The $10 for a 10 minute back and neck massage ...


In [12]:
nearby_tips.to_csv('competitors.csv')

### Now it is your turn to discover interesting search results using Foursquare
You are expected to read through the API documentation and customize the searches.  